In [2]:
import cv2
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt

In [61]:
# Inicializar Mediapipe
mp_face_mesh = mp.solutions.face_mesh
#face_mesh = mp_face_mesh.FaceMesh()
mp_drawing = mp.solutions.drawing_utils
#mp_drawing_styles = mp.solutions.drawing_styles

with mp_face_mesh.FaceMesh(
    static_image_mode = True,
    max_num_faces=1,
    min_detection_confidence=0.5) as face_mesh:

    #Cargar la imagen
    image_path = r"c:\Users\Lucia\Downloads\sad_prueba.jpg"
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error al cargar la imagen desde {image_path}. Verifica la ruta y asegúrate de que el archivo exista.")
    else:
        # Convertir la imagen a RGB (Mediapipe usa RGB)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen con Mediapipe
        results = face_mesh.process(image_rgb)
        
        # Convertir de nuevo a BGR para mostrar usando OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        print("Face Landmarks: ", results.multi_face_landmarks)

        if results.multi_face_landmarks is not None:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image_bgr,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=1, circle_radius=1))
                
                # Obtener coordenadas de puntos específicos
                landmarks = face_landmarks.landmark
                h, w, _ = image.shape
                
                # Puntos específicos (ajustar los índices si es necesario)
                points = [
                    (landmarks[33].x * w, landmarks[33].y * h),  # ojo izquierdo
                    (landmarks[263].x * w, landmarks[263].y * h),  # ojo derecho
                    (landmarks[2].x * w, landmarks[2].y * h),     # Base Nariz
                    (landmarks[152].x * w, landmarks[152].y * h),  # Mentón
                    (landmarks[0].x * w, landmarks[0].y * h),  # labio superior
                    (landmarks[17].x * w, landmarks[17].y * h)  # labio inferior
                ]
 
                # Convertir a enteros
                points = [(int(x), int(y)) for x, y in points]
 
                x1, y1 = points[0]
                x2, y2 = points[1]
                x3, y3 = points[2]
                x4, y4 = points[3]
                x5, y5 = points[4]
                x6, y6 = points[5]

                # Punto medio entre los ojos
                x_mid = (x1 + x2) / 2
                y_mid = (y1 + y2) / 2

                eye_distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                eye_to_nose = abs(y3 - y_mid)
                nose_to_chin = np.sqrt((x4 - x3)**2 + (y4 - y3)**2)
                mouth_distance = np.sqrt((x6 - x5)**2 + (y6 - y5)**2)
                mouth_to_chin = np.sqrt((x6 - x4)**2 + (y6 - y4)**2)

                proportion_eye_chin = nose_to_chin / eye_to_nose
                proportion_mouth_chin = mouth_to_chin / mouth_distance

                emotions_detected = []

                # Evaluar la primera condición
                if (0.94 <= proportion_eye_chin <= 1.34) and (1.41 <= proportion_mouth_chin <= 2.43):
                    emotions_detected.append("Angry")
                    print("Emotion: Angry")

                # Evaluar la segunda condición
                elif (0.96 <= proportion_eye_chin <= 1.33) and (1.53 <= proportion_mouth_chin <= 1.97):
                    emotions_detected.append("Sad")
                    print("Emotion: Sad")

                # Evaluar la tercera condición
                elif (1.04 <= proportion_eye_chin <= 1.83) and (0.76 <= proportion_mouth_chin <= 1.46):
                    emotions_detected.append("Happy")
                    print("Emotion: Happy")

                # Si no se detectó ninguna emoción, asignar "Other"
                if not emotions_detected:
                    emotions_detected.append("Other")
                    print("Is other emotion")

                # Mostrar todas las emociones detectadas
                print("Emotions detected:", emotions_detected)
        
                # Mostrar resultados
                print(f'Proporción ojo-mentón: {proportion_eye_chin}')
                print(f'Proporción boca-mentón: {proportion_mouth_chin}')

                print(eye_distance, eye_to_nose, nose_to_chin,mouth_distance, )
                
                # Dibujar los puntos y las líneas
                cv2.circle(image_bgr, (x1, y1), 5, (0, 0, 255), -1)
                cv2.circle(image_bgr, (x2, y2), 5, (0, 0, 255), -1)
                cv2.circle(image_bgr, (x3, y3), 5, (0, 0, 255), -1)
                cv2.circle(image_bgr, (x4, y4), 5, (0, 0, 255), -1)
                cv2.circle(image_bgr, (x5, y5), 5, (0, 0, 255), -1)
                cv2.circle(image_bgr, (x6, y6), 5, (0, 0, 255), -1)
                cv2.circle(image_bgr, (int(x_mid), int(y_mid)), 5, (0, 0, 255), -1)
                cv2.line(image_bgr, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.line(image_bgr, (x3, y3), (x4, y4), (255, 0, 0), 2)
                cv2.line(image_bgr, (x4, y4), (x6, y6), (255, 0, 0), 2)
                cv2.line(image_bgr, (int(x_mid), int(y_mid)), (x3, y3), (255, 0, 0), 2)
                cv2.putText(image_bgr, f"Emotion: {emotions_detected}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)

        # Mostrar la imagen
        cv2.imshow('Face Mesh', image_bgr)
        cv2.waitKey(0)
    cv2.destroyAllWindows()



Face Landmarks:  [landmark {
  x: 0.5210035443305969
  y: 0.5270545482635498
  z: -0.01897994987666607
}
landmark {
  x: 0.520603597164154
  y: 0.4868620038032532
  z: -0.03767251968383789
}
landmark {
  x: 0.5201771259307861
  y: 0.49889466166496277
  z: -0.01923665776848793
}
landmark {
  x: 0.5131598114967346
  y: 0.4457138776779175
  z: -0.029176389798521996
}
landmark {
  x: 0.5203695297241211
  y: 0.4741155803203583
  z: -0.040235958993434906
}
landmark {
  x: 0.5196978449821472
  y: 0.45731472969055176
  z: -0.03766442462801933
}
landmark {
  x: 0.5180664658546448
  y: 0.4175291657447815
  z: -0.019542401656508446
}
landmark {
  x: 0.4670325815677643
  y: 0.4161156117916107
  z: 0.00580279715359211
}
landmark {
  x: 0.5174124836921692
  y: 0.3922010064125061
  z: -0.01559440791606903
}
landmark {
  x: 0.5173144340515137
  y: 0.3764343857765198
  z: -0.017402473837137222
}
landmark {
  x: 0.5161625742912292
  y: 0.3145313560962677
  z: -0.01008576899766922
}
landmark {
  x: 0.521